## Ambulance Call Data Preprocessing

## Introduction
This notebook focuses on **preprocessing and feature engineering** for the NYC ambulance daily call dataset.  
The goal of this step is to transform the raw data into a clean, structured, and model-ready format.  
Since the dataset spans only **10 months**, it is especially important to preserve as much information as possible while carefully creating features that capture meaningful patterns in ambulance demand.

## Objectives
1. **Data Cleaning**  
   - Ensure consistent formats for dates and categorical variables.  
   - Handle missing values without dropping rows, since our dataset is small.  

2. **Feature Engineering**  
   - Create lag-based features (e.g., yesterday’s calls, last week’s calls).  
   - Add difference features to capture daily and weekly changes.  
   - Generate rolling averages to smooth out noise and highlight short/medium-term trends.  

3. **Encoding Categorical Variables**  
   - One-hot encode boroughs and day-of-week.  
   - Retain binary indicators for weekends and holidays.  

4. **Final Dataset Preparation**  
   - Produce a fully preprocessed dataset with no missing values.  
   - Save clean features for use in the **modeling notebook**.  

## Why Preprocessing Matters
Ambulance demand is influenced by both **time-dependent patterns** (lags, rolling averages) and **external factors** (borough, weather, holidays).  
Careful preprocessing ensures that these signals are captured in a way that machine learning models (such as **XGBoost**) can effectively use.  



# Import libraries

In [1]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt 
import seaborn as sns

# Datetime handling 
from datetime import datetime

# Preprocessing and Encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load data 

In [2]:
df_processed = pd.read_csv('daily_ambulance_calls.csv')
df_processed.head()

,Unnamed: 0,date,borough,daily_call_vol,mean_severity_level,max_severity_level,min_severity,day_of_week,is_weekend,is_holiday,lag_1,lag_7,precp,snowfall,temp
0,0,2024-07-01,Bronx,1039,4.216554,1,8,0,0,0,1021.557047,1020.561644,0.4,0.0,70.34
1,1,2024-07-02,Bronx,950,4.195789,1,8,1,0,0,1039.000000,1020.561644,0.0,0.0,71.87
2,2,2024-07-03,Bronx,1002,4.229541,1,8,2,0,0,950.000000,1020.561644,0.0,0.0,71.87
3,3,2024-07-04,Bronx,907,4.060639,1,8,3,0,1,1002.000000,1020.561644,1.3,0.0,75.47
4,4,2024-07-05,Bronx,1099,4.270246,1,8,4,0,0,907.000000,1020.561644,15.6,0.0,78.71


In [3]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           1495 non-null   int64  
 1   date                 1495 non-null   object 
 2   borough              1495 non-null   object 
 3   daily_call_vol       1495 non-null   int64  
 4   mean_severity_level  1495 non-null   float64
 5   max_severity_level   1495 non-null   int64  
 6   min_severity         1495 non-null   int64  
 7   day_of_week          1495 non-null   int64  
 8   is_weekend           1495 non-null   int64  
 9   is_holiday           1495 non-null   int64  
 10  lag_1                1495 non-null   float64
 11  lag_7                1495 non-null   float64
 12  precp                1495 non-null   float64
 13  snowfall             1495 non-null   float64
 14  temp                 1495 non-null   float64
dtypes: float64(6), int64(7), object(2)
mem

## More Feature Engineering

To improve the predictive power of our model on ambulance daily call volumes, we create several **time-series derived features**. These features help the model capture trends, seasonality, and recent changes in call patterns.

### 1. Lag Features
- **lag_1, lag_2, lag_7**: Represent the call volume from yesterday, 2 days ago, and one week ago.  
-  In time series, the past strongly influences the present. Including lagged values helps the model learn short-term and weekly patterns.

### 2. Lag Differences
- **lag_diff_1 = lag_1 – lag_2**: Captures day-to-day change in call volume.  
- **week_diff = lag_7 – lag_1**: Captures the difference between yesterday and last week (weekly change).  
- Ambulance demand can shift abruptly due to events (holidays, weather, outbreaks). Change features help detect these dynamics better than absolute values.

### 3. Rolling Averages
- **roll_mean_3**: 3-day rolling average of call volume.  
- **roll_mean_7**: 7-day rolling average of call volume.  
- Motivation: Rolling windows smooth out noise and capture short-term (3-day) and medium-term (weekly) trends, which improves stability in the predictions.

### 4. Handling Missing Values
- Lag and rolling features naturally create missing values at the start of each borough's time series.  
- Instead of dropping rows (which would reduce our already small dataset), we apply:
  - **Forward Fill (ffill)**: Uses the most recent valid value.  
  - **Median Fill**: For the very first rows where forward fill is not possible.




In [4]:
# Converting date to datetime 
df_processed.date = pd.to_datetime(df_processed.date, errors='coerce')

# Create lag_2 feature
df_processed['lag_2'] = df_processed.groupby('borough')['daily_call_vol'].shift(2)

# create lag_diff_1 : Day to day change in call volume (yesterday vs 2 days ago)
df_processed['lag_diff_1'] = df_processed.lag_1 - df_processed.lag_2

# create week_diff : weekly change (yesterday vs last week same day)
df_processed['week_diff'] = df_processed.lag_7 - df_processed.lag_1

# create roll_3 : 3 day rolling average of call volume 
df_processed['roll_3'] =(
                        df_processed.groupby('borough')['daily_call_vol']. 
                         apply(lambda x : x.shift(1).rolling(3, min_periods=1).mean()).
                         reset_index(level=0, drop=True)
                        )   
    
# create roll_7 : 7 day rolling average of call volume 
df_processed['roll_7'] = (
                          df_processed.groupby('borough')['daily_call_vol'].
                           apply(lambda x : x.shift(1).rolling(7, min_periods=1).mean()).
                            reset_index(level=0, drop=True)
                            )
df_processed.iloc[0:100:7]

,Unnamed: 0,date,borough,daily_call_vol,mean_severity_level,max_severity_level,min_severity,day_of_week,is_weekend,is_holiday,lag_1,lag_7,precp,snowfall,temp,lag_2,lag_diff_1,week_diff,roll_3,roll_7
0,0,2024-07-01,Bronx,1039,4.216554,1,8,0,0,0,1021.557047,1020.561644,0.4,0.0,70.34,NaN,NaN,-0.995403,NaN,NaN
7,7,2024-07-08,Bronx,1134,4.194885,1,7,0,0,0,1009.000000,1039.000000,0.0,0.0,82.04,1049.0,-40.0,30.000000,1052.333333,1007.857143
14,14,2024-07-15,Bronx,1230,4.162602,1,8,0,0,0,1070.000000,1134.000000,1.6,0.0,82.04,995.0,75.0,64.000000,1028.666667,1072.857143
21,21,2024-07-22,Bronx,1137,4.186456,1,8,0,0,0,1091.000000,1230.000000,3.3,0.0,76.19,994.0,97.0,139.000000,1048.000000,1082.000000
28,28,2024-07-29,Bronx,1077,4.319406,1,7,0,0,0,1002.000000,1137.000000,3.8,0.0,76.37,980.0,22.0,135.000000,977.000000,1024.000000
35,35,2024-08-05,Bronx,1089,4.234160,1,7,0,0,0,941.000000,1077.000000,0.0,0.0,79.16,993.0,-52.0,136.000000,1012.333333,1029.142857
42,42,2024-08-12,Bronx,1075,4.301395,1,8,0,0,0,1020.000000,1089.000000,0.4,0.0,70.25,941.0,79.0,69.000000,1007.333333,996.714286
49,49,2024-08-19,Bronx,1011,4.281899,1,8,0,0,0,937.000000,1075.000000,10.1,0.0,75.38,877.0,60.0,138.000000,954.666667,998.428571
56,56,2024-08-26,Bronx,1068,4.299625,1,8,0,0,0,949.000000,1011.000000,0.1,0.0,75.47,955.0,-6.0,62.000000,962.666667,962.428571
63,63,2024-09-02,Bronx,924,4.232684,1,8,0,0,1,932.000000,1068.000000,0.0,0.0,70.97,901.0,31.0,136.000000,937.666667,964.285714


In [5]:
# Check for mising values
df_processed.isna().sum()

Unnamed: 0              0
date                    0
borough                 0
daily_call_vol          0
mean_severity_level     0
max_severity_level      0
min_severity            0
day_of_week             0
is_weekend              0
is_holiday              0
lag_1                   0
lag_7                   0
precp                   0
snowfall                0
temp                    0
lag_2                  10
lag_diff_1             10
week_diff               0
roll_3                  5
roll_7                  5
dtype: int64

In [6]:
# Check for mising values by index
nan_rows = df_processed[df_processed[['lag_2','lag_diff_1','week_diff','roll_3','roll_7']].isna().any(axis=1)].index
print(nan_rows)

Index([0, 1, 299, 300, 598, 599, 897, 898, 1196, 1197], dtype='int64')


## Handle missing values
Lag and rolling features naturally create missing values at the start of each borough's time series.
Instead of dropping rows (which would reduce our already small dataset), we apply:
Forward Fill (ffill): Uses the most recent valid value.
Median Fill: For the very first rows where forward fill is not possible.


In [7]:
# Forward fill by borough 
df_processed = df_processed.groupby('borough').apply(lambda x : x.ffill()).reset_index(drop=True)

# Median fill for the first row, where ffill is not posiible 
df_processed = df_processed.fillna(df_processed.median(numeric_only=True))

df_processed.iloc[0:50:7]

/var/folders/f3/hdbg9f_j0cn761_jtcpg4pkr0000gn/T/ipykernel_37673/441946333.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_processed = df_processed.groupby('borough').apply(lambda x : x.ffill()).reset_index(drop=True)


,Unnamed: 0,date,borough,daily_call_vol,mean_severity_level,max_severity_level,min_severity,day_of_week,is_weekend,is_holiday,lag_1,lag_7,precp,snowfall,temp,lag_2,lag_diff_1,week_diff,roll_3,roll_7
0,0,2024-07-01,Bronx,1039,4.216554,1,8,0,0,0,1021.557047,1020.561644,0.4,0.0,70.34,984.0,-3.0,-0.995403,996.666667,1001.714286
7,7,2024-07-08,Bronx,1134,4.194885,1,7,0,0,0,1009.000000,1039.000000,0.0,0.0,82.04,1049.0,-40.0,30.000000,1052.333333,1007.857143
14,14,2024-07-15,Bronx,1230,4.162602,1,8,0,0,0,1070.000000,1134.000000,1.6,0.0,82.04,995.0,75.0,64.000000,1028.666667,1072.857143
21,21,2024-07-22,Bronx,1137,4.186456,1,8,0,0,0,1091.000000,1230.000000,3.3,0.0,76.19,994.0,97.0,139.000000,1048.000000,1082.000000
28,28,2024-07-29,Bronx,1077,4.319406,1,7,0,0,0,1002.000000,1137.000000,3.8,0.0,76.37,980.0,22.0,135.000000,977.000000,1024.000000
35,35,2024-08-05,Bronx,1089,4.234160,1,7,0,0,0,941.000000,1077.000000,0.0,0.0,79.16,993.0,-52.0,136.000000,1012.333333,1029.142857
42,42,2024-08-12,Bronx,1075,4.301395,1,8,0,0,0,1020.000000,1089.000000,0.4,0.0,70.25,941.0,79.0,69.000000,1007.333333,996.714286
49,49,2024-08-19,Bronx,1011,4.281899,1,8,0,0,0,937.000000,1075.000000,10.1,0.0,75.38,877.0,60.0,138.000000,954.666667,998.428571


In [8]:
# Check for missing values 
df_processed.isna().sum()

Unnamed: 0             0
date                   0
borough                0
daily_call_vol         0
mean_severity_level    0
max_severity_level     0
min_severity           0
day_of_week            0
is_weekend             0
is_holiday             0
lag_1                  0
lag_7                  0
precp                  0
snowfall               0
temp                   0
lag_2                  0
lag_diff_1             0
week_diff              0
roll_3                 0
roll_7                 0
dtype: int64

## Encoding Categorical Features

We are applying one-hot encoding to the `borough` column only.  

- `borough` is a nominal categorical feature, so one-hot encoding allows the model to treat each borough as a separate category.  
- All other categorical or integer-coded features (`day_of_week`, `is_weekend`, `is_holiday`, `max_severity_level`, `min_severity`) are left as numeric, since XGBoost can handle them directly.

In [13]:
# one hot encode `borough` by applying pandas get_dummies()
# drop_first=True, to avoid multicollinearity
df_final = pd.get_dummies(df_processed, columns=['borough'], drop_first=True, dtype= int) 

df_final.head()

,Unnamed: 0,date,daily_call_vol,mean_severity_level,max_severity_level,min_severity,day_of_week,is_weekend,is_holiday,lag_1,...,temp,lag_2,lag_diff_1,week_diff,roll_3,roll_7,borough_Brooklyn,borough_Manhattan,borough_Queens,borough_Staten Island
0,0,2024-07-01,1039,4.216554,1,8,0,0,0,1021.557047,...,70.34,984.0,-3.0,-0.995403,996.666667,1001.714286,0,0,0,0
1,1,2024-07-02,950,4.195789,1,8,1,0,0,1039.000000,...,71.87,984.0,-3.0,-18.438356,1039.000000,1039.000000,0,0,0,0
2,2,2024-07-03,1002,4.229541,1,8,2,0,0,950.000000,...,71.87,1039.0,-89.0,70.561644,994.500000,994.500000,0,0,0,0
3,3,2024-07-04,907,4.060639,1,8,3,0,1,1002.000000,...,75.47,950.0,52.0,18.561644,997.000000,997.000000,0,0,0,0
4,4,2024-07-05,1099,4.270246,1,8,4,0,0,907.000000,...,78.71,1002.0,-95.0,113.561644,953.000000,974.500000,0,0,0,0


## Train/Test Split and Scaling Decisions

### 1. Train / Validation / Test Split  

For this project, the dataset (July 2024 – April 2025) is divided chronologically to respect the **time-series nature** of EMS call data. Random splits are avoided since they can leak future information into the past.  

- **Train (~7 months: July 2024 → Jan/Feb 2025):** Used to fit the model and learn historical patterns.  
- **Validation (~2 months: Feb/March 2025):** Held out for hyperparameter tuning and model selection.  
- **Test (~1 month: April 2025):** Final unseen data to evaluate how the model performs on truly new future calls.  

This setup ensures the model mimics a real-world forecasting scenario where we only have past data available to predict future demand.  

### 2. Scaling / Standardization
- We are **not applying StandardScaler** to the numeric features.  
- Reason: XGBoost is a **tree-based model** that is **scale-invariant**, meaning it can handle features in different ranges without standardization.  
- Scaling is unnecessary and does not improve model performance in this case.  


## Train / Validation / Test Split  

In [14]:
# Confirm date to datetime format and sort 
df_final['date'] = pd.to_datetime(df_final.date)
df_final = df_final.sort_values('date')

# Define cutoff dates
train_end = '2025-01-31'
val_end = '2025-03-31'

# Chronological split
train = df_final[df_final.date<=train_end]
val = df_final[(df_final.date>train_end) & (df_final.date<=val_end)]
test = df_final[df_final.date>val_end]

print('Train shape :', train.shape)
print('Validation shape :', val.shape)
print('Test shape :', test.shape)

Train shape : (1075, 23)
Validation shape : (295, 23)
Test shape : (125, 23)


In [15]:
# X_train and y_train
X_train = train.drop(columns=['date','daily_call_vol'])
y_train = train['daily_call_vol']

# X_val and y_val
X_val = val.drop(columns=['date','daily_call_vol'])
y_val = val['daily_call_vol']

# X_test and y_test
X_test = test.drop(columns=['date','daily_call_vol'])
y_test = test['daily_call_vol']

print('X_train shape:', X_train.shape, 'y_train shape:', y_train.shape)
print('X_val shape:', X_val.shape, 'y_val shape:', y_val.shape)
print('X_test shape:', X_test.shape, 'y_test shape:', y_test.shape)

X_train shape: (1075, 21) y_train shape: (1075,)
X_val shape: (295, 21) y_val shape: (295,)
X_test shape: (125, 21) y_test shape: (125,)


## Save processed data

In [16]:
# Save full processed Dataframe to csv
df_final.to_csv('df_final.csv', index = False)

# Save Train nad Test dataset to csv
X_train.to_csv('X_train.csv', index = False)
X_test.to_csv('X_test.csv', index = False)
X_val.to_csv('X_val.csv', index = False)
y_val.to_csv('y_val.csv', index = False)
y_train.to_csv('y_train.csv', index = False)
y_test.to_csv('y_test.csv', index = False)

# Summary

In this preprocessing notebook, we prepared the ambulance call dataset for modeling by first converting the date column to datetime format and sorting records chronologically within each borough. We engineered time-based features such as lag variables (lag_1, lag_2, lag_7), short-term and weekly differences (lag2 - lag1, lag7 - lag1), and rolling averages (rolling_3, rolling_7) to capture both recent history and smoothed trends. Additional categorical features like day of week, weekend, and holiday indicators were also included. Missing values were handled through forward fill where possible, with median imputation applied to remaining cases to minimize data loss. We then applied one-hot encoding to the borough column while retaining other severity-related codes as numeric values suitable for XGBoost. Finally, we structured the data for chronological split, ensuring that the temporal nature of the dataset is respected and preventing leakage. With these steps, the dataset is now clean, feature-rich, and ready for borough-level and overall modeling.